# Brief Intro to PyTorch

In [ ]:
import os, math, time, warnings, textwrap, sys
from pathlib import Path

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

try:
    from scipy.special import kv, kvp   # Bessel K(v, x) and ∂/∂v K(v, x)
    SCIPY_OK = True
except Exception as e:
    SCIPY_OK = False
    warnings.warn("SciPy not found. Custom Bessel example will be CPU-only or skipped.")

torch.manual_seed(0)
np.random.seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
print("PyTorch:", torch.__version__)

## 1) Why autograd (vs NumPy) & backprop

NumPy is great for arrays but **doesn't track computation graphs**. If you want gradients, you must do calculus by hand.  
PyTorch **records ops**, so calling `.backward()` gives you **exact vector–Jacobian products** using optimized backprop.

Two key advantages:

**1. Automatic Differentiation (Autograd)**
- In NumPy, you have to hand-code derivatives. This is slow, error-prone, and hard to maintain.
- In PyTorch, the computation graph is recorded automatically. Calling `.backward()` computes gradients *exactly* with efficient backpropagation.
- This means we can train deep models with millions of parameters without ever touching calculus.

**2. Optimized GPU Kernels**
- PyTorch uses high-performance backends (MKL, cuBLAS, cuDNN).  
- Matrix multiplies, convolutions, and reductions run *orders of magnitude* faster than naive NumPy on CPU.  
- On GPU, PyTorch can be 10–100× faster than NumPy on the same task.  

Together: PyTorch makes it possible to write **concise, research-friendly code** that is also **fast enough for production training**.


Below: the same tiny regression loss in NumPy (manual grad) vs PyTorch (autograd).


In [ ]:
# Toy: y_hat = w * x ; loss = (y_hat - y)^2

# NumPy (manual gradient)
x_np = np.array([1.0, 2.0, 3.0])
y_np = np.array([2.0, 4.1, 5.9])
w_np = 0.0

# forward
yhat_np = w_np * x_np
loss_np = np.mean((yhat_np - y_np)**2)

# manual grad dL/dw = 2 * mean((w*x - y) * x)
grad_w_np = 2.0 * np.mean((yhat_np - y_np) * x_np)

print(f"[NumPy] loss={loss_np:.4f}, grad_w={grad_w_np:.4f}")

# PyTorch (autograd)
x = torch.tensor(x_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32)
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

yhat = w * x
loss = torch.mean((yhat - y)**2)
loss.backward()
print(f"[Torch]  loss={loss.item():.4f}, grad_w={w.grad.item():.4f}")

In [ ]:
import numpy as np
import torch, time

# -------------------------
# Example 1: gradient by hand vs autograd
# -------------------------
x_np = np.linspace(-2, 2, 100)
w_np = 3.0
y_np = w_np * x_np**2

# manual gradient of loss wrt w
# loss = mean(w * x^2)
grad_w_np = np.mean(x_np**2)
print(f"[NumPy manual] grad wrt w = {grad_w_np:.4f}")

# PyTorch version
x = torch.linspace(-2, 2, 100, dtype=torch.float32)
w = torch.tensor(3.0, requires_grad=True)
y = w * x**2
loss = y.mean()
loss.backward()
print(f"[PyTorch autograd] grad wrt w = {w.grad.item():.4f}")

# -------------------------
# Example 2: performance (matrix multiply)
# -------------------------
N = 2000

A_np = np.random.randn(N, N).astype(np.float32)
B_np = np.random.randn(N, N).astype(np.float32)

A_torch = torch.from_numpy(A_np)
B_torch = torch.from_numpy(B_np)

# NumPy
t0 = time.time()
C_np = A_np @ B_np
t1 = time.time()
print(f"[NumPy] {t1 - t0:.3f} sec")

# PyTorch
t0 = time.time()
C_torch = A_torch @ B_torch
t1 = time.time()
print(f"[PyTorch] {t1 - t0:.3f} sec")

## 2) Torch tensors: creation, shapes, dtypes, devices

Key tips:
- Keep **model & inputs on the same device** (`cpu` or `cuda`).
- Move **once per step** (avoid ping-ponging between CPU/GPU).
- Use `.to(device)` or `.cuda()` / `.cpu()`.
- Prefer **float32** (training) or **bfloat16**/**float16** with **AMP**.

In [ ]:
# creation
a = torch.zeros((2, 3), dtype=torch.float32)
b = torch.ones_like(a)
c = torch.rand((2, 3))                  # U[0,1)
d = torch.randn((3, 3))                 # N(0,1)
e = torch.arange(0, 10).reshape(2, 5)

# shape/dtype/device
print(a.shape, a.dtype, a.device)
print(f'a is {a}')
print(f'b is {b}')
print(f'c is {c}')
print(f'd is {d}')
print(f'e is {e}')

# device moves
a_gpu = a.to(device)
d_gpu = d.to(device, non_blocking=True)  # non_blocking requires pinned CPU memory when coming from DataLoader

# simple math
z = torch.matmul(d_gpu, torch.eye(3, device=device))
print(f'z is {z}')
print("z device:", z.device, "z shape:", z.shape)

## 3) Tracking autograd vs not

- `requires_grad=True` starts tracking.
- **Default**: tensors created from ops on tracked tensors also track.
- Turn off with:
  - `with torch.no_grad():` (training or inference; faster, saves memory)
  - `with torch.inference_mode():` (inference only; even leaner)
- Break gradient flow with `.detach()`.
- Non-scalar losses need `loss.backward(gradient=...)`.

In [ ]:
w = torch.tensor(2.0, requires_grad=True)
x = torch.tensor(3.0)

# Tracks: (w * x)^2
y = (w * x) ** 2
y.backward()
print("dw after backward:", w.grad.item())

# Stop tracking
with torch.no_grad():
    y2 = (w * x) ** 2  # no graph
# or
y3 = (w.detach() * x) ** 2  # explicitly breaks graph

The Difference between `detach()` vs `item()`

In [ ]:
# Start with a scalar tensor that requires gradients
x = torch.tensor(2.0, requires_grad=True)
y = x**2 + 3*x

# Case 1: .item()
# ----------------
# .item() extracts a standard Python number (float) from a one-element tensor.
# Useful for logging/printing, but breaks the computation graph (no gradients).
print("Using .item():")
print("y as Python number:", y.item())
print("Type:", type(y.item()))

# Case 2: .detach()
# -----------------
# .detach() creates a new tensor that shares the same data
# but is disconnected from the computation graph.
# Useful if you want the tensor for further computation, but without gradient tracking.
z = y.detach()
print("\nUsing .detach():")
print("z (detached tensor):", z)
print("z requires_grad?", z.requires_grad)

# Demonstrating difference in backprop
y.backward()  # backprop works because y is connected to x
print("\nGradient in x.grad after y.backward():", x.grad)

try:
    z.backward()
except RuntimeError as e:
    print("\nBackprop through detached tensor fails:")
    print(e)

## 4) When standard ops aren't enough: Custom `autograd.Function`

Example: **Matérn kernel** uses the modified Bessel function of the second kind \(K_\nu\), which PyTorch doesn't natively expose.  
We can **wrap SciPy** (CPU) for the forward and supply our **own backward**. Caveats:

- Keep **dtype/device** consistent (we'll use `float64` for stability).
- Moving to NumPy implies CPU; be explicit on device transfers.
- Numerical finite-differences for gradients are slower/approximate → good for correctness, not for production speed.

In [ ]:
def is_positive_definite(matrix: torch.Tensor) -> bool:
    """Check if a matrix is positive definite."""
    try:
        torch.linalg.cholesky(matrix)
        return True
    except RuntimeError:
        return False

if not SCIPY_OK:
    warnings.warn("SciPy not available: skipping custom BesselKFunction demonstration.")

class BesselKFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, v: torch.Tensor, x: torch.Tensor):
        """
        v, x: float64 tensors (can be broadcastable).
        We'll compute on CPU via SciPy, then return to x.device.
        """
        if not SCIPY_OK:
            raise RuntimeError("SciPy is required for this example.")

        # Save for backward (save tensors as-is; do not detach here)
        ctx.save_for_backward(v, x)

        # Move to CPU numpy
        v_cpu = v.detach().cpu().numpy()
        x_cpu = x.detach().cpu().numpy()

        out = kv(v_cpu, x_cpu)  # SciPy: elementwise modified Bessel K
        out_t = torch.from_numpy(np.asarray(out)).to(dtype=torch.float64, device=x.device)
        return out_t

    @staticmethod
    def backward(ctx, grad_output: torch.Tensor):
        v, x = ctx.saved_tensors
        if not SCIPY_OK:
            raise RuntimeError("SciPy is required for this example.")

        # Finite-diff wrt x
        eps_x = 1e-12
        v_cpu = v.detach().cpu().numpy()
        x_cpu = x.detach().cpu().numpy()

        k_plus  = kv(v_cpu, x_cpu + eps_x)
        k_minus = kv(v_cpu, x_cpu - eps_x)
        dK_dx = (k_plus - k_minus) / (2 * eps_x)

        # Exact derivative wrt v via kvp (SciPy)
        dK_dv = kvp(v_cpu, x_cpu)

        dK_dx_t = torch.from_numpy(np.asarray(dK_dx)).to(dtype=torch.float64, device=x.device)
        dK_dv_t = torch.from_numpy(np.asarray(dK_dv)).to(dtype=torch.float64, device=v.device)

        # Chain rule
        grad_v = grad_output * dK_dv_t
        grad_x = grad_output * dK_dx_t
        return grad_v, grad_x  # matches (v, x) order

def matern_kernel(pairwise_distances: torch.Tensor,
                  length_scale: torch.Tensor,
                  nu: torch.Tensor,
                  sigma: torch.Tensor,
                  epsilon: float = 1e-9) -> torch.Tensor:
    """
    Matérn covariance with broadcasting support.
    All inputs should be float64 to match our Bessel implementation.
    """
    # ensure float64 for stability with special functions
    dtype = torch.float64
    P = pairwise_distances.to(dtype)
    L = length_scale.to(dtype)
    N = nu.to(dtype)
    S = sigma.to(dtype)
    sigma2 = S**2

    # avoid exactly 0.5 to dodge special-case branches
    N = torch.where(N == 0.5, N + epsilon, N)

    # scaled distances
    scaled = torch.sqrt(2.0 * N) * (P / L)
    # clamp for numeric stability
    scaled = torch.clamp(scaled, min=1e-10, max=1e6)

    # Bessel term (custom autograd)
    K = BesselKFunction.apply(N, scaled)

    # front factor: 2^{1-ν} / Γ(ν)
    scaling = (2.0 ** (1.0 - N)) / torch.exp(torch.lgamma(N))

    cov = sigma2 * scaling * (scaled ** N) * K
    # exact diag at zero distance
    cov = torch.where(P == 0, sigma2, cov)
    return cov

In [ ]:
# Demo: using the custom BesselKFunction inside the Matérn kernel

# Small set of "locations"
X = torch.linspace(0, 1, 5, dtype=torch.float64)
pairwise_dist = torch.cdist(X.unsqueeze(1), X.unsqueeze(1))

# Parameters (require gradients so we can test backprop)
length_scale = torch.tensor(0.2, dtype=torch.float64, requires_grad=True)
nu          = torch.tensor(0.9, dtype=torch.float64, requires_grad=True)
sigma       = torch.tensor(1.0, dtype=torch.float64, requires_grad=True)

# Build covariance matrix
K = matern_kernel(pairwise_dist, length_scale, nu, sigma)
print("Covariance matrix:\n", K)
print("Positive definite?", is_positive_definite(K))

# Toy "loss" = log determinant + trace (GP-style terms)
loss = torch.logdet(K) + torch.trace(K)
print("Loss value:", loss.item())

# Backward: should give gradients wrt hyperparams
loss.backward()
print("dL/d length_scale:", length_scale.grad.item())
print("dL/d nu:", nu.grad.item())
print("dL/d sigma:", sigma.grad.item())

## 5) Optimization strategies

- Separate **learning rates** per parameter group when scales differ.  
- **Early stopping** with patience.  
- **Reduce LR on plateau** (or manual reduction).  
- **Gradient clipping** to tame exploding grads.  
- **Anomaly detection** while debugging.

Below uses **Adagrad** with different per-param LRs (as in your snippet).

In [ ]:
# Example scalar params to fit (toy)
alpha_i = torch.tensor(0.1,  dtype=torch.float64, requires_grad=True)
nu_i    = torch.tensor(0.9,  dtype=torch.float64, requires_grad=True)
sigma_i = torch.tensor(1.0,  dtype=torch.float64, requires_grad=True)

optimizer = torch.optim.Adagrad([
    {'params': [alpha_i], 'lr': 0.0003},
    {'params': [nu_i],    'lr': 0.008},
    {'params': [sigma_i], 'lr': 0.25},
], lr_decay=0, weight_decay=0, eps=1e-15)

# Optional: scheduler (manual below, but here's a standard one)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

tolerance = 1e-15
patience = 25
best_loss = float('inf')
epochs_no_improve = 0
max_lr_reductions = 3
reductions = 0

gradient_clip = 5.0
number_of_cycles = 100
steps_per_batch = 5

# Dummy batches (replace with your X_groups, Y_groups)
X_groups = [torch.linspace(0, 1, 50, dtype=torch.float64)]
Y_groups = [torch.sin(X_groups[0])]

def toy_nll(alpha, nu, sigma, X, Y):
    # This is a stand-in for your Matérn-based NLL.
    # Replace with your real loss.
    pred = sigma * torch.exp(-alpha * (X - 0.5)**2)
    return torch.mean((pred - Y)**2) + 0.01*(alpha**2 + nu**2 + sigma**2)

for epoch in range(number_of_cycles):
    total_loss = 0.0
    for Xb, Yb in zip(X_groups, Y_groups):
        for _ in range(steps_per_batch):
            optimizer.zero_grad(set_to_none=True)
            loss = toy_nll(alpha_i, nu_i, sigma_i, Xb, Yb)
            loss.backward()
            nn.utils.clip_grad_norm_([alpha_i, nu_i, sigma_i], gradient_clip)
            optimizer.step()
            total_loss += loss.item()

    avg_loss = total_loss / (len(X_groups) * steps_per_batch)
    improved = avg_loss + tolerance < best_loss
    if improved:
        best_loss = avg_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # Manual LR reduce on plateau
    if epochs_no_improve >= patience and reductions < max_lr_reductions:
        for pg in optimizer.param_groups:
            pg['lr'] *= 0.1
        reductions += 1
        epochs_no_improve = 0

    # scheduler.step(avg_loss)  # if using ReduceLROnPlateau
    if epoch % 10 == 0:
        print(f"epoch {epoch:03d} | loss {avg_loss:.6e} | lr {[pg['lr'] for pg in optimizer.param_groups]}")

## 6) DTypes under GPU constraints

Common training defaults:
- **float32** (safe)
- **bfloat16 / float16 + AMP** (Automatic Mixed Precision) → faster & less memory
- Model weights often **float16/bfloat16**, activations **float16/bfloat16**, **master weights** kept in float32 (optimizers).

For **very tight memory** (many teams do this):
- **int8 / int4 weight quantization** for *inference*
- **activation checkpointing** to trade compute for memory
- **gradient accumulation** to simulate larger batch sizes
- **bf16** preferred over fp16 on modern GPUs (better range, no loss scaling)

Below: AMP training pattern.

In [ ]:
use_amp = (device.type == "cuda")
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

model = nn.Sequential(nn.Linear(32, 64), nn.ReLU(), nn.Linear(64, 10)).to(device)
opt = optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

X = torch.randn(256, 32, device=device)
y = torch.randint(0, 10, (256,), device=device)

for step in range(10):
    opt.zero_grad(set_to_none=True)
    with torch.cuda.amp.autocast(enabled=use_amp):
        logits = model(X)
        loss = loss_fn(logits, y)
    scaler.scale(loss).backward()
    scaler.unscale_(opt)
    nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    scaler.step(opt)
    scaler.update()

## 7) Debugging mode

- `torch.autograd.set_detect_anomaly(True)` pinpoints the exact bad op (slow; use temporarily).
- Catch **NaN/Inf** with hooks or checks.
- Use `gradcheck` for custom ops.
- Profile with `torch.autograd.profiler` or `torch.profiler`.

In [ ]:
torch.autograd.set_detect_anomaly(True)  # SLOW — only enable while debugging

def nan_hook(grad):
    if torch.isnan(grad).any() or torch.isinf(grad).any():
        print("Detected NaN/Inf in gradient!")
    return grad

# Attach to a parameter for demo
param = nn.Parameter(torch.randn(()), requires_grad=True)
param.register_hook(nan_hook)

loss = (param * float('nan'))**2  # force NaN
try:
    loss.backward()
except Exception as e:
    print("Backward failed (as expected in demo):", e)

## 8) Estimating time

- Measure **avg batch time** × remaining batches → ETA.
- Use a lightweight **moving average**.
- Add a **tqdm** progress bar if you like.

In [ ]:
from collections import deque # doubled ended queue which automatically drops the oldest items

def run_with_eta(n_steps=200, window=20):
    times = deque(maxlen=window)
    t0 = time.perf_counter()
    for step in range(1, n_steps + 1):
        # simulate work
        time.sleep(0.005)
        t1 = time.perf_counter()
        times.append(t1 - t0)
        t0 = t1

        avg = sum(times) / len(times)
        remaining = n_steps - step
        eta_s = remaining * avg
        if step % 25 == 0 or step == n_steps:
            print(f"step {step}/{n_steps} | avg_step {avg*1000:.2f} ms | ETA {eta_s:.2f} s")

run_with_eta()

## 9) Conversion: NumPy, Pandas, Torch

- `torch.from_numpy(np_array)` (shares memory; CPU only)
- `tensor.numpy()` (CPU only; shares memory)
- For GPU tensors, first `.cpu()`.
- Pandas → Tensor: convert columns to NumPy, then `from_numpy`.
- Zero-copy interop: **DLPack** (`torch.utils.dlpack`).

Beware: **shared memory** means in-place changes reflect on both sides.

In [ ]:
# NumPy -> Torch
np_arr = np.random.randn(3, 4).astype(np.float32)
t_from_np = torch.from_numpy(np_arr)         # shares memory (CPU)
print("shares memory:", t_from_np.data_ptr() == torch.from_numpy(np_arr).data_ptr())

# Torch -> NumPy
np_back = t_from_np.numpy()                  # zero-copy (CPU)
t_gpu = torch.randn(3, 4, device=device)
np_from_gpu = t_gpu.detach().cpu().numpy()   # copy

# Pandas -> Torch
df = pd.DataFrame({"x": np.random.randn(5), "y": np.random.randn(5)})
X_t = torch.from_numpy(df.values).float()    # (5, 2)

# DLPack (zero-copy advanced)
capsule = torch.utils.dlpack.to_dlpack(t_from_np)
t2 = torch.utils.dlpack.from_dlpack(capsule)  # shares storage

## 10) Memory profiler & run tracking

- CUDA memory: `torch.cuda.memory_allocated()`, `max_memory_allocated()`, `reset_peak_memory_stats()`.
- Model summary: `torch.cuda.memory_summary()`.
- Wall-clock: `time.perf_counter()`.
- Lightweight experiment log: append CSV rows with config/metrics.

In [ ]:
def memory_snapshot(tag="snapshot"):
    if device.type == "cuda":
        torch.cuda.synchronize()
        alloc = torch.cuda.memory_allocated() / 1024**2
        peak  = torch.cuda.max_memory_allocated() / 1024**2
        print(f"[{tag}] CUDA alloc={alloc:.1f} MB | peak={peak:.1f} MB")
    else:
        print(f"[{tag}] CPU mode; use tracemalloc or psutil for details.")

# demo
if device.type == "cuda":
    torch.cuda.reset_peak_memory_stats()
    a = torch.randn(1024, 1024, device=device)
    memory_snapshot("after big tensor")

# simple CSV logger
from datetime import datetime
log_path = Path("runs_log.csv")
def log_run(**kwargs):
    row = {"timestamp": datetime.now().isoformat(), **kwargs}
    df = pd.DataFrame([row])
    if log_path.exists():
        df.to_csv(log_path, mode="a", header=False, index=False)
    else:
        df.to_csv(log_path, index=False)
    return row

_ = log_run(model="demo-mlp", lr=1e-3, best_loss=0.1234, epochs=10, notes="sanity check")
print("Logged to", log_path.resolve())

In [ ]:
%load_ext memory_profiler

In [ ]:
%reload_ext memory_profiler

# Allocate a big tensor
%memit a = torch.randn(10000, 10000)

# Free it
del a